## Libraries


In [ ]:
# !pip install -r ~/code/benitomartin/FoodScore/requirements.txt

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanIoU


import pickle

## Data import

In [ ]:
coord = pd.DataFrame()

for i in range(1, 257, 1):
    path = f"../raw_data/UECFOOD256/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [ ]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [ ]:
coord.shape

In [ ]:
coord.head()

In [ ]:
coord = coord.rename(columns={"img": "img_name"})

In [ ]:
coord.head()

### Rescaling and Normalization

In [ ]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD256/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [ ]:
rescaled_coord.head()

### add image paths

In [ ]:
list_paths = [f"../raw_data/UECFOOD256/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [ ]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [ ]:
rescaled_coord.head()

### balancing Dataset

In [ ]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [ ]:
classes = list(set(rescaled_coord.label))

In [ ]:
df = rebalancing(rescaled_coord, classes, av_number= 200, random_state=1)

In [ ]:
rescaled_coord[rescaled_coord['label']==100].shape

In [ ]:
df[df['label']==100].shape

### load downscaled pictures into array

In [ ]:
from tqdm.auto import tqdm

In [ ]:
df.head()

In [ ]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

In [ ]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

In [ ]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [ ]:
trainImages, testImages,trainLabels, testLabels,trainBBoxes, testBBoxes,trainPaths, testPaths=\
train_test_split(images,
                 labels,
                 bboxes,
                 paths,
                 test_size=0.20,
                 random_state=42)

## Model

In [ ]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=layers.Input(shape=(224, 224, 3)))


vgg.trainable = False

flatten = vgg.output
flatten = layers.Flatten()(flatten)

bboxHead = layers.Dense(128, activation="relu")(flatten)
bboxHead = layers.Dense(64, activation="relu")(bboxHead)
bboxHead = layers.Dense(32, activation="relu")(bboxHead)
bboxHead = layers.Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)

softmaxHead = layers.Dense(512, activation="relu")(flatten)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(512, activation="relu")(softmaxHead)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(257, activation="softmax", name="class_label")(softmaxHead)


In [ ]:
model = Model(
    inputs=vgg.input,
    outputs=(bboxHead, softmaxHead))

In [ ]:
losses = {
    "class_label": 'sparse_categorical_crossentropy',
    "bounding_box": "mse"
}

In [ ]:
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

In [ ]:
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}

In [ ]:
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

In [ ]:
metrics = {
    "class_label": "sparse_categorical_accuracy",
    "bounding_box": MeanIoU(num_classes=len(lb.classes_))
}

In [ ]:
opt = Adam(0.001)


model.compile(loss=losses, 
              optimizer=opt, 
              metrics=["accuracy",'mse'], 
              loss_weights=lossWeights)

print(model.summary())

In [ ]:
es = EarlyStopping(monitor = 'accuracy',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [ ]:
history = model.fit(
    trainImages,
    trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=32,
    epochs=1000,
    verbose=1,
    )